In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import zscore
import re

df = pd.read_csv("../../Analyse_Sentiments/friends_dialogues_final.csv", encoding='utf-8')

def clean_line(line):
    """
    Enlève les annotations/didascalies, les ensemble de mots entre parenthèses
    Mets en minuscule
    Enlève les caractères spéciaux
    Split la ligne en mots
    :param line: réplique de dialogue
    :return: liste de mots
    """
    line = re.sub(r'\([^)]*\)', '', line)
    # lower + clean special char + split
    return re.sub(r'[^\w\s]', '', line.lower()).split()

### TEST GRAPHE AVEC EN PLUS NOMBRE DE FOIS SON NOM DANS UN FICHIER (pas strandardisé par nombre d'ép)

In [3]:
def recherche_pic_de_popularite_personnage():
    saisons = sorted(df['season'].unique())
    characters = ['monica', 'joey', 'chandler', 'phoebe', 'ross', 'rachel']

    mentions_per_season = pd.DataFrame(0, index=saisons, columns=characters)

    # parcours de toutes les lignes pour compter les mentions par saison
    for _, row in df.iterrows():
        saison = row['season']
        ligne = clean_line(row['line'])
        speaker = row['character'].lower()

        for mot in ligne:
            if mot in characters and mot != speaker:
                # mot est donc un personnage ici
                mentions_per_season.at[saison, mot] += 1
    
    # total des mentions par saison afin de normaliser
    total_lines_per_season = mentions_per_season.sum(axis=1)

    # normalisation par le total des répliques dans la saison
    mentions_per_season_norm = mentions_per_season.div(total_lines_per_season, axis=0)

    # total de répliques par saison
    total_lines_per_season = df.groupby('season').size()

    # nombre de répliques par personnage et saison
    count = df.groupby(['character', 'season']).size().reset_index(name='line_count')

    # ajout du total des répliques dans la saison correspondante
    count['total_lines_in_season'] = count['season'].map(total_lines_per_season)

    # normalisation par le total des répliques de la saison
    count['line_count_norm'] = count['line_count'] / count['total_lines_in_season']

    pivot_norm = count.pivot(index='character', columns='season', values='line_count_norm').fillna(0)
    mentions_per_season_norm = mentions_per_season_norm.T

    mainData = pd.read_csv("friends_episode_names.csv", encoding='utf-8')

    # ajouter colonnes booléennes (mention ou non dans le titre)
    for character in characters:
        mainData[character] = mainData['episodename'].str.contains(character, case=False, na=False).astype(int)

    # grouper par saison et faire la somme
    mention_counts = mainData.groupby('season', as_index=True)[characters].sum().reset_index()

    mentions_in_titles = mention_counts.T

    mentions_in_titles_zscore = pd.DataFrame(
        zscore(mentions_in_titles, axis=1, nan_policy='omit'),
        index=mentions_in_titles.index,
        columns=mentions_in_titles.columns
    )

    # Z-scores
    mentions_zscore = pd.DataFrame(
        zscore(mentions_per_season_norm, axis=1, nan_policy='omit'),
        index=mentions_per_season_norm.index,
        columns=mentions_per_season_norm.columns
    )

    line_zscore = pd.DataFrame(
        zscore(pivot_norm, axis=1, nan_policy='omit'),
        index=pivot_norm.index,
        columns=pivot_norm.columns
    )

    for character in characters:
        # affichage
        plt.figure(figsize=(10, 6))
        plt.title(f"Tendance de mise en avant de {character.capitalize()} (répliques et mentions) par saison (normalisé par z-score)")
        plt.xlabel("Saison")
        plt.ylabel("Personnage")
        plt.tight_layout()
        plt.plot(line_zscore.T[character.capitalize()], marker='o', label='Fréquence de répliques')
        plt.plot(mentions_zscore.T[character], marker='o', label='Fréquence de mentions')
        plt.plot(mentions_in_titles_zscore.T[character], marker='o', label='Fréquence de mentions dans le titre')
        plt.plot((line_zscore.T[character.capitalize()]+mentions_zscore.T[character])/2, marker='o', label='Moyenne des répliques et mentions')
        plt.legend()
        plt.show()

recherche_pic_de_popularite_personnage()

TypeError: unsupported operand type(s) for /: 'str' and 'int'